## Tool for making Sample Manager outputs more user-friendly

### Leila Barker, October 2024

#### Summary:
* Save Sample Manager output file to this location (W:\TD&R\Python Tools\Sample Manager data cleanup tool\Tool inputs)
    * Ensure that the file is saved as an Excel Workbook file (not an older version such as Excel 97-2003 Workbook)
    * The tool will import the most recently modified file in this folder if there is more than one file
* Run the following cells in order
* The output file will be saved in the folder W:\TD&R\Python Tools\Sample Manager data cleanup tool\Tool inputs as "SM_cleaned_today's date"

#### Step 1: Import data, delete unauthorized data, clean up DateTimes

In [110]:
# Import the data from the most recent Excel file in the "Tool inputs" folder

import os
import pandas as pd
from datetime import datetime

folder_path = 'W:\TD&R\Python Tools\Sample Manager data cleanup tool\Tool inputs'

files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx') or f.endswith('.xls')]
files_full_path = [os.path.join(folder_path, f) for f in files]

# Find the latest file by modification time and load into a dataframe
latest_file = max(files_full_path, key=os.path.getmtime)
latest_file_name = os.path.basename(latest_file) # Name of the latest file

# Load the latest file into a dataframe
raw_df = pd.read_excel(latest_file)

print(f"Input file path: {latest_file}")
print(f"Input file name: {latest_file_name}")

# Delete any data that does not have the status "Authorised"
raw_df.drop(raw_df.columns[0], axis=1, inplace=True) # Deletes the first column (standard SM export -- not needed)
raw_df = raw_df.loc[raw_df['Result Status'] == 'Authorised'] # Retains only data that has been authorised (has passed final QA/QC)

# Convert datetimes to standard format; add a column for Date without time

temp_df = pd.DataFrame()
temp_df = raw_df
temp_df['DateTime'] = pd.to_datetime(temp_df['Sampled Date'], errors='coerce', format='%m/%d/%Y %I:%M %p')
temp_df['JustDate'] = pd.to_datetime(temp_df['DateTime']).dt.strftime('%Y-%m-%d')
raw_df['Date'] = temp_df['JustDate']

# Move DateTime next to Date
columns = raw_df.columns.tolist()
date_index = columns.index('Sampled Date')  # Get the index of the 'Date' column
columns.remove('Date')
columns.insert(date_index + 1, 'Date') # Insert 'Date' right after 'Sampled Date'
columns.remove('DateTime')
columns.insert(date_index + 1, 'DateTime') # Insert 'DateTime' two after 'Sampled Date'
raw_df = raw_df[columns] # Reorder the DataFrame based on the new column order
raw_df = raw_df.drop(columns=['Sampled Date']) # Delete 'Sampled Date' column
raw_df.drop(columns=raw_df.columns[-1], axis=1, inplace=True) # Delete the last column (temporary column called 'JustDate')

raw_df.head()

Input file path: W:\TD&R\Python Tools\Sample Manager data cleanup tool\Tool inputs\tmpCE5C.xlsx
Input file name: tmpCE5C.xlsx


,Sample Number,Sampling Point,Sample Name,DateTime,Date,Analysis,Component,Qualifiers,Result,Units,Result Status,Comments
0,488348,Miscellaneous Sample,FG GW well,2024-02-13,2024-02-13,ICANION-S,NO2 (Report),NaN,<0.050,ppm_wt_v,Authorised,NaN
1,488348,Miscellaneous Sample,FG GW well,2024-02-13,2024-02-13,ICANION-S,NO2NO3 (Report),NaN,1.41,ppm_wt_v,Authorised,NaN
2,488348,Miscellaneous Sample,FG GW well,2024-02-13,2024-02-13,ICANION-S,NO3 (Report),NaN,1.40,ppm_wt_v,Authorised,NaN
3,488348,Miscellaneous Sample,FG GW well,2024-02-13,2024-02-13,NH3OPO4-S,NH3 (Report),NaN,0.078,ppm_wt_v,Authorised,NaN
4,488348,Miscellaneous Sample,FG GW well,2024-02-13,2024-02-13,NH3OPO4-S,o-PO4 (Report),NaN,0.268,ppm_wt_v,Authorised,NaN


#### Step 2 (Optional): delete qualified data and/or select qualifiers (this is a placeholder for any modifications the user wants to make prior to exporting the data)

In [113]:
# Optional: delete qualified data (note: this deletes all data with something in the "Qualifiers" field;
# if desired, can modify to just delete data with certain qualifiers [e.g., U])

# We could also use this section to create a new field that indicates which parameters were flagged or commented on, and the content of those flags/comments, prior to the creation of the final table.

raw_df = raw_df[raw_df['Qualifiers'].isna()]

#### Step 3: Create a pivot table with each row representing a single sample

In [116]:
# Create a pivot table of the data

pivot_df = raw_df.copy()

# Create a new field, "Parameter", combining the "Analysis" and "Component" fields
pivot_df['Parameter'] = pivot_df['Analysis'] + ' ' + pivot_df['Component'] + ' (' + pivot_df['Units'] + ')'

# Create a pivot table
pivot_df = pivot_df.pivot_table(index=['Sample Number', 'Sampling Point', 'Sample Name', 'Date', 'DateTime'], columns = 'Parameter', values='Result', aggfunc='first').reset_index()

#Simplify the parameter names
# Function to convert column names
def convert_column_name(col_name):
    if "(Report)" in col_name:
        # Extract the part just before "(Report)"
        part_before_report = col_name.split("(Report)")[0].strip().split()
        # Get the last word (the parameter) and the prefix (if any)
        parameter = part_before_report[-1]
        prefix = ' '.join(part_before_report[:-1])  # Join everything before the last word

        # Determine the concentration unit
        if "(ppm_wt_v)" in col_name:
            concentration_unit = " (mg/L)"
        elif "(ppb_wt_v)" in col_name:
            concentration_unit = " (ug/L)"
        else:
            concentration_unit = ""

        # Determine the suffix based on conditions
        if "-S" in prefix and prefix != "ICANION-S":
            return f"{parameter} (S){concentration_unit}"
        elif "-T" in prefix:
            return f"{parameter} (T){concentration_unit}"
        else:
            return f"{parameter}{concentration_unit}"

    return col_name  # Return unchanged if it doesn't match the pattern


# Apply the conversion function to all column names
pivot_df.columns = [convert_column_name(col) for col in pivot_df.columns]

pivot_df.head()


,Sample Number,Sampling Point,Sample Name,Date,DateTime,CBOD (mg/L),Chloroph-A-C (ug/L),Chloroph-A-U (ug/L),Pheoph-A (ug/L),COD (S) (mg/L),...,Tl (T) (ug/L),V (T) (ug/L),Zn (T) (ug/L),NH3 (S) (mg/L),o-PO4 (S) (mg/L),NPOC (T) (mg/L),TKN (mg/L),TP (mg/L),TSS (mg/L),TVSS (mg/L)
0,488348,Miscellaneous Sample,FG GW well,2024-02-13,2024-02-13 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.078,0.268,NaN,0.372,0.299,NaN,NaN
1,488349,Miscellaneous Sample,"LTW soil 12""",2024-02-15,2024-02-15 08:20:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.014,0.148,NaN,0.349,0.179,NaN,NaN
2,488350,Miscellaneous Sample,"1A-1B berm soil 12""",2024-02-15,2024-02-15 08:30:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.138,0.042,NaN,1.63,0.179,NaN,NaN
3,488351,Miscellaneous Sample,"SE lake soil 18""",2024-02-15,2024-02-15 08:35:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.41,0.010,NaN,7.21,0.422,NaN,NaN
4,488352,Miscellaneous Sample,"Upland soil near VFW 18""",2024-02-15,2024-02-15 08:15:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.286,0.015,NaN,1.22,0.116,NaN,NaN


#### Step 4. Export the data as an Excel workbook

In [119]:
# Get the current date in order to export the cleaned file with a timestamp
current_year = datetime.now().year
today = datetime.today().strftime('%Y-%m-%d')

# Export the file
output_directory = r'W:\TD&R\Python Tools\Sample Manager data cleanup tool\Tool outputs'
latest_file_name_no_ext = os.path.splitext(latest_file_name)[0]
filename = f'{latest_file_name_no_ext}_{today}.xlsx'
output_filepath = os.path.join(output_directory, filename)
pivot_df.to_excel(output_filepath)
print(f'File saved at: {output_filepath}')

File saved at: W:\TD&R\Python Tools\Sample Manager data cleanup tool\Tool outputs\tmpCE5C_2024-10-15.xlsx
